In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import data_processing as dp
from scipy import signal
from scipy.stats import skew
import seaborn as sns
import matplotlib.pyplot as plt
import pywt
import os
import time
import h5py
from itertools import combinations

In [ ]:
for dirname, _, filenames in os.walk('.\data'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

In [ ]:
path2 = '.\data\正常\G11_Walking_trial_4_emg.csv'

In [52]:
Data = pd.read_csv('./processed data/dataframe_W256_S64_DWTLmax_samelabel_sc.csv')

# read file name of data with various Labels
df = pd.read_csv('./useful_data_label.csv',index_col=0) 
drop = 'G07_Freezing_Trial1_trial_1_emg.csv'
drop2= 'P812_M050_2_B_FoG_trial_2_emg.csv'
drop3= 'P812_M050_2_B_FoG_trial_1_emg.csv'
ind_drop = (df.columns!=drop)# & (df.columns!=drop2) & (df.columns!=drop3)
# read file name of data with only label 0
df2 = pd.read_csv('./unuseful_data_label.csv',index_col=0)
# read some of the data with only label 0
df3 = pd.read_csv('./data/file_name.txt',header=None)

files = np.concatenate([np.array(df.columns),np.array(df3.loc[:,0])])
ind = Data.File.isin(files)
ind2 = Data.File == drop
#ind = (Data.File != drop) & (Data.File != drop2)
Data_sel = Data[ind]
Data_rest = Data[ind2]
#ind2 = Data_rest.File == drop
#Data_rest = Data_rest[ind2]
#Data_rest

In [93]:
feature_col = ['_IEMG','_MAV','_SSI','_VAR','_RMS',
               '_WL','_ZC','_SSC','_WAMP','_skew','_Acti','_AR','_HIST','_MDF']

feature_all = Data_sel.iloc[:,1:-1]
#ind_temp1 = feature_all.columns.str.contains('_mDWT')
#ind_temp2 = feature_all.columns.str.contains('_Coe')
#ind_temp3 = feature_all.columns.str.contains('_Scale')
#ind_temp4 = feature_all.columns.str.contains('_HIST')
#ind_temp = ind_temp1|ind_temp2|ind_temp3|ind_temp4
ind_temp = feature_all.columns.str.contains('_Acti')
#feature = feature_all.loc[:,~ind_temp]
feature = Data_sel.iloc[:,1:-1]
#feature = Data_sel.iloc[:,:-2]
y = Data_sel.Label
feature2_all = Data_rest.iloc[:,1:-1]
#feature2 = feature2_all.loc[:,~ind_temp]
feature2 = Data_rest.iloc[:,1:-1]
#feature2 = Data_rest.iloc[:,:-2]
y2 = Data_rest.Label

In [162]:
feature = pd.DataFrame()
feature2 = pd.DataFrame()
y = pd.DataFrame()
y2 = pd.DataFrame()
m = 0
for i in set(Data_sel.File):
    ind = Data_sel.File==i
    temp = Data_sel[ind]
    #if i != 'G08_FoG_2_trial_1_emg.csv':
        #feature = pd.concat([feature,temp.iloc[:,1:-1]])
        #y = pd.concat([y,temp.iloc[:,0]])
    #else:
    for j in set(temp.Label):
        ind2 = temp.Label == j
        temp2 = temp[ind2]
        l = len(temp2)
        feature = pd.concat([feature,temp2.iloc[:int(0.8*l),1:-1]])
        y = pd.concat([y,temp2.iloc[:int(0.8*l),0]])
        feature2 = pd.concat([feature2,temp2.iloc[int(0.8*l):,1:-1]])
        y2 = pd.concat([y2,temp2.iloc[int(0.8*l):,0]])
y = np.array(y)[:,0]
y2 = np.array(y2)[:,0]
#ind_temp = feature.columns.str.contains('_mDWT')
#feature = feature.loc[:,~ind_temp]
#feature2 = feature2.loc[:,~ind_temp]

In [124]:
#ind_temp1 = feature_all.columns.str.contains('_MF')
#ind_temp2 = feature_all.columns.str.contains('_MDF')
#ind_temp3 = feature_all.columns.str.contains('_AR')
#ind_temp4 = feature_all.columns.str.contains('_MNF')
#ind_temp = ind_temp1|ind_temp2|ind_temp3|ind_temp4
#feature = feature.loc[:,~ind_temp]
#feature2 = feature2.loc[:,~ind_temp]

In [163]:
feature2

,LEFT_TA_IEMG,LEFT_TS_IEMG,LEFT_BF_IEMG,LEFT_RF_IEMG,RIGHT_TA_IEMG,RIGHT_TS_IEMG,RIGHT_BF_IEMG,RIGHT_RF_IEMG,LEFT_TA_SSI,LEFT_TS_SSI,...,RIGHT_BF_mDWT6,RIGHT_BF_mDWT7,RIGHT_RF_mDWT0,RIGHT_RF_mDWT1,RIGHT_RF_mDWT2,RIGHT_RF_mDWT3,RIGHT_RF_mDWT4,RIGHT_RF_mDWT5,RIGHT_RF_mDWT6,RIGHT_RF_mDWT7
14494,54.495407,41.838260,16.685675,119.245790,136.606220,15.870949,57.179630,52.119680,23.876139,13.751545,...,0.333921,0.199823,41.699135,27.142443,9.567805,4.691112,1.127269,0.526081,0.310089,0.097274
14495,63.584263,51.623400,17.644413,133.229200,117.331640,19.025179,52.941120,48.827780,29.531832,17.729313,...,0.124742,0.074639,46.585384,33.111549,15.584126,8.166800,5.065614,2.192267,1.282910,0.469431
14496,62.807045,62.645560,21.137068,143.969420,103.604040,17.383175,48.306545,37.493343,29.937807,26.072136,...,0.483473,0.196852,33.096977,22.685007,10.285604,4.408403,2.025230,0.828235,0.442531,0.232302
14497,61.515120,47.318073,62.659695,139.217450,89.551180,17.632595,36.355846,30.375315,27.534600,17.827032,...,0.486986,0.268595,29.801613,19.772127,11.282631,6.269420,3.268324,1.412688,0.756074,0.323066
14498,48.220585,40.681900,86.668440,140.539630,89.653465,17.877747,28.514355,27.504467,19.109505,15.961396,...,0.164356,0.079370,24.088203,14.794522,6.858083,2.881250,1.461718,0.659022,0.342953,0.183987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,47.052890,40.982760,80.379090,71.945190,81.862625,76.749030,32.495710,33.282760,23.372030,19.965347,...,0.564377,0.265130,22.907181,10.707895,5.132853,2.635980,0.775645,0.324570,0.185962,0.092924
84,70.285400,24.368923,65.337105,60.989666,83.281130,63.575268,50.984562,69.853836,38.233658,6.818824,...,0.089207,0.015426,40.678551,20.532425,12.888038,6.459993,3.246606,1.451732,0.777140,0.384506
85,55.515575,26.936937,67.611600,60.874900,90.378430,66.137920,49.313670,68.964874,27.921959,8.506439,...,1.764694,0.877015,33.579285,14.800857,5.733666,2.417968,0.496807,0.214147,0.012236,0.004933
88,66.963390,35.998043,109.073320,93.629906,53.432440,57.514908,30.049881,34.337666,38.418472,13.120143,...,0.117951,0.049956,22.185177,12.946556,7.217345,4.386374,2.870004,1.276046,0.617854,0.308553


In [ ]:
path = './processed data/data_set_after_window_withoutSC.hdf5'
with h5py.File(path,'r') as f:
    x = f['cwt_data'][...]
    y = f['label2'][...]

In [1]:
import imp
imp.reload(dp)

NameError: name 'dp' is not defined

In [3]:
import ctypes
player = ctypes.windll.kernel32
player.Beep(1000,200)

1

In [128]:
(Data_rest.File=='P812_M050_2_B_FoG_trial_1_emg.csv').sum()

0

In [ ]:
width = 256
threshold_WAMP = 30
threshold_ZC = 0
threshold_SSC = 1
bins=9
bound = 70
HIST_range = (-bound,bound)

In [4]:
path = './processed data/data_set_after_window_W128_S64_sameLabel_rect.hdf5'
with h5py.File(path,'r') as f:
    x = f['data'][...]
    y = f['label2'][...]
#feature = dp.generate_feature(x,threshold_WAMP=threshold_WAMP,
#                              threshold_ZC=threshold_ZC,
#                              threshold_SSC=threshold_SSC,
#                              bins=bins,ranges=HIST_range)
#feature2 = dp.generate_feature(x2)
player.Beep(1000,200)

1

In [ ]:
with h5py.File('./processed data/nfeatures_W256_S64_WAMP30.hdf5','r') as f:
    feature = f['features'][...]
    y = f['labels'][...]

In [ ]:
feature2,y2 = dp.pipeline_feature(path2,width=256,stride=64,
                                  scaler=False,
                                  threshold_WAMP=threshold_WAMP,
                                  threshold_ZC=threshold_ZC,
                                  threshold_SSC=threshold_SSC,
                                  bins=bins,ranges=HIST_range)

In [ ]:
with h5py.File('./processed data/nfeatures_rest_W256_S64.hdf5','r') as f:
    feature2 = f['features'][...]
    y2 = f['labels'][...]

In [ ]:
sc = StandardScaler(with_mean=True)
feature_sc = sc.fit_transform(feature)
feature2_sc = sc.transform(feature2)

# Model

In [4]:
import tensorflow as tf
from tensorflow.keras import layers,Model,callbacks,regularizers,models
from sklearn.model_selection import train_test_split,cross_validate
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder,normalize,MinMaxScaler
from sklearn.decomposition import PCA
from imblearn.over_sampling import SMOTE,BorderlineSMOTE,ADASYN,SVMSMOTE

In [5]:
tf.executing_eagerly()

True

In [174]:
def train_model(model,feature,y,binary=True,file=None):
    if binary:
        ind = ((y==0)|(y==1)|(y==2)|(y==6))
        #ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y[ind].copy()
        ind1 = ((y_01==1)|(y_01==2)|(y_01==6))
        y_01[ind1] = 1
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:1,1:5}
    else:
        ind = ((y==1)|(y==2)|(y==6))
        #ind = ((y==1)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:5,1:1,2:1}#{0:2,1:1,2:10,3:2}
    x_full,x_test,y_full,y_test = train_test_split(np.array(feature)[ind,:],y_01,
                                                   test_size=0.2,
                                                   random_state=123,
                                                   shuffle=True)
    x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,
                                                       test_size=0.25,
                                                       random_state=555,
                                                       shuffle=True)
    
    #sm = BorderlineSMOTE(random_state=50,kind='borderline-2')
    #sm = SMOTE(random_state=50)
    #print(y_full.shape)
    #x_full,y_full = sm.fit_resample(x_full,y_full)
    #print(y_full_n.shape)
    sc = StandardScaler(with_mean=True)
    #sc = MinMaxScaler()
    x_train = sc.fit_transform(x_full)
    pca = PCA(n_components=100)
    #x_train = pca.fit_transform(x_train)
    #x_valid = sc.transform(x_valid)
    x_test = sc.transform(x_test)
    #x_test = pca.transform(x_test)
    #x_train = x_full
    
    early_stopping = callbacks.EarlyStopping(patience = 20,
                                             monitor = 'val_loss', 
                                             restore_best_weights=True)
    history = model.fit(x_train,y_full,validation_data=(x_test,y_test),
                        epochs=300,batch_size=32,class_weight=cw,
                        callbacks=[early_stopping],
                        shuffle=True)
    
    if binary:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #test = metrics.accuracy_score(y_test,y_pred_t>0.5)
        
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(y_valid,np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))
        #train = metrics.accuracy_score(y_full,y_pred_ta>0.5)
        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
        
        #print('train: \n',metrics.confusion_matrix(y_full,y_pred_ta>0.5))
        #print('test: \n',metrics.confusion_matrix(y_test,y_pred_t>0.5))

    else:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
    print('test:%f'%test)
    #print('valid:%f'%valid)
    print('train:%f'%train)
    if file != None:
        model.save_model(file)
    return train,test,sc,pca

def test_model(model,feature,y,sc,pca,binary=True):
    if binary:
        ind = ((y==0)|(y==1)|(y==2)|(y==3)|(y==4)|(y==6))
        #ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y[ind].copy()
        ind1 = ((y_01==1)|(y_01==2)|(y_01==6))
        y_01[ind1] = 1
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:1,1:5}
    else:
        ind = ((y==1)|(y==2)|(y==6))
        #ind = ((y==1)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:5,1:1,2:1}#{0:2,1:1,2:10,3:2}

    #print(y_01)
    feature=sc.transform(feature[ind])
    #feature = feature[ind]
    #feature=pca.transform(feature)
    y_pred=model.predict(feature)
    test = metrics.accuracy_score(np.argmax(y_01,axis=1),np.argmax(y_pred,axis=1))
    #test = metrics.accuracy_score(y_01,y_pred>0.5)
    
    print('acc:%f'%test)
    print(metrics.confusion_matrix(np.argmax(y_01,axis=1),np.argmax(y_pred,axis=1)))
    #print(metrics.confusion_matrix(y_01,y_pred>0.5))
    return test

def sparse_cost_sensitive_loss (y_true,y_pred):
    #cost_matrix = tf.constant([[0,1.,1,1.],
    #              [2,0,5,5],
    #              [1,1,0,1],
    #              [1.,2.,1,0]])
    cost_matrix = tf.constant([[0,2.,2],
                  [1,0,1],
                  [1.0,1.,0]])
    #cost_matrix = tf.constant([[0,1.],
    #              [5.,0]])
    batch_cost_matrix = tf.nn.embedding_lookup(cost_matrix, tf.argmax(y_true,axis=1))
    eps = 1e-6
    probability = tf.clip_by_value(y_pred, eps, 1-eps)
    cost_values = tf.math.log(1-probability)*batch_cost_matrix
    loss = tf.reduce_mean(-tf.reduce_sum(cost_values, axis=1))
    return loss

## ANN

In [6]:
ind = ((y==1)|(y==2)|(y==3)|(y==6))
#ind = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
ind_f = [0,1,6,42,46,57,62]
#y_01 = y[ind].copy()
#y_01[y_01==1]=0
#y_01[y_01==2]=1
#y_01[y_01==3]=2
#y_01[y_01==6]=3
y_01 = y[ind].copy()
#y_01[ind] = 1
oh_ec = OneHotEncoder()
y_oh = oh_ec.fit_transform(y_01[:,np.newaxis]).toarray()
x_full,x_test,y_full,y_test = train_test_split(feature.loc[ind,:],y_01,test_size=0.2,random_state=123,shuffle=False)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555,shuffle=True)

In [6]:
input_ = layers.Input(shape=feature.shape[1:])#feature.shape[1:]
l1 = layers.Dense(128,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(input_)
#drop1 = layers.Dropout(0.2)(l1)
bn1 = layers.BatchNormalization()(l1)
l2 = layers.Dense(64,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(bn1)
#drop2 = layers.Dropout(0.2)(l2)
bn2 = layers.BatchNormalization()(l2)
l3 = layers.Dense(32,activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                 )(bn2)
#drop3 = layers.Dropout(0.2)(l3)
bn3 = layers.BatchNormalization()(l3)
l4 = layers.Dense(16,activation='elu',
                 #kernel_regularizer = regularizers.l2(0.001),
                 )(bn3)
#drop4 = layers.Dropout(0.2)(l4)
#l5 = layers.Dense(8,activation='relu')(drop4)
#drop5 = layers.Dropout(0.5)(l5)
output = layers.Dense(3,activation='softmax')(l4)
model = Model(inputs=[input_],outputs=[output])

In [171]:
model = models.Sequential()
model.add(layers.BatchNormalization())
model.add(layers.Dense(128,#activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                       #use_bias=False
                 ))
#model.add(layers.Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.Activation('selu'))
model.add(layers.Dense(64,#activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                      # use_bias=False
                 ))
#model.add(layers.Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.Activation('selu'))
model.add(layers.Dense(32,#activation='elu',
                  #kernel_initializer='lecun_normal',
                  #kernel_regularizer = regularizers.l2(0.001),
                       #use_bias=False
                 ))
#model.add(layers.Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.Activation('selu'))
model.add(layers.Dense(16,#activation='elu',
                 #kernel_regularizer = regularizers.l2(0.001),
                       #use_bias=False
                 ))
#model.add(layers.Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.Dense(3,activation='softmax'))

In [172]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

# Train model

In [173]:
#sc = StandardScaler(with_mean=True)
#feature_sc = sc.fit_transform(feature)
#feature2_sc = sc.transform(feature2)
#pca = PCA(n_components=160,copy=True)
#feature_pca = pca.fit_transform(feature)
#feature2_pca = pca.transform(feature2)
train,test,sc,pca = train_model(model,feature,np.array(y),False)
#rest = test_model(model,feature2,np.array(y2),sc)
#acc = [train,valid,test,rest]

Epoch 1/300
213/213 [==============================] - 2s 8ms/step - loss: 0.6574 - accuracy: 0.7215 - val_loss: 0.5657 - val_accuracy: 0.7670
Epoch 2/300
213/213 [==============================] - 1s 5ms/step - loss: 0.5432 - accuracy: 0.7757 - val_loss: 0.5497 - val_accuracy: 0.7705
Epoch 3/300
213/213 [==============================] - 1s 6ms/step - loss: 0.5064 - accuracy: 0.7889 - val_loss: 0.5070 - val_accuracy: 0.7811
Epoch 4/300
213/213 [==============================] - 1s 5ms/step - loss: 0.4665 - accuracy: 0.8049 - val_loss: 0.5013 - val_accuracy: 0.7881
Epoch 5/300
213/213 [==============================] - 1s 5ms/step - loss: 0.4479 - accuracy: 0.8131 - val_loss: 0.4648 - val_accuracy: 0.8081
Epoch 6/300
213/213 [==============================] - 1s 5ms/step - loss: 0.4215 - accuracy: 0.8224 - val_loss: 0.4489 - val_accuracy: 0.8151
Epoch 7/300
213/213 [==============================] - 1s 4ms/step - loss: 0.4036 - accuracy: 0.8321 - val_loss: 0.4397 - val_accuracy: 0.8140

Epoch 58/300
213/213 [==============================] - 1s 4ms/step - loss: 0.0852 - accuracy: 0.9686 - val_loss: 0.2813 - val_accuracy: 0.9137
Epoch 59/300
213/213 [==============================] - 1s 4ms/step - loss: 0.0707 - accuracy: 0.9745 - val_loss: 0.3250 - val_accuracy: 0.8991
Epoch 60/300
213/213 [==============================] - 1s 5ms/step - loss: 0.0873 - accuracy: 0.9690 - val_loss: 0.2886 - val_accuracy: 0.9126
Epoch 61/300
213/213 [==============================] - 1s 5ms/step - loss: 0.0689 - accuracy: 0.9756 - val_loss: 0.2624 - val_accuracy: 0.9102
Epoch 62/300
213/213 [==============================] - 1s 6ms/step - loss: 0.0605 - accuracy: 0.9772 - val_loss: 0.2778 - val_accuracy: 0.9196
Epoch 63/300
213/213 [==============================] - 1s 6ms/step - loss: 0.0700 - accuracy: 0.9737 - val_loss: 0.2669 - val_accuracy: 0.9108
Epoch 64/300
213/213 [==============================] - 1s 6ms/step - loss: 0.0775 - accuracy: 0.9718 - val_loss: 0.2584 - val_accuracy:

Epoch 115/300
213/213 [==============================] - 1s 4ms/step - loss: 0.0310 - accuracy: 0.9890 - val_loss: 0.2914 - val_accuracy: 0.9167
Epoch 116/300
213/213 [==============================] - 1s 4ms/step - loss: 0.0436 - accuracy: 0.9861 - val_loss: 0.2349 - val_accuracy: 0.9313
Epoch 117/300
213/213 [==============================] - 1s 4ms/step - loss: 0.0427 - accuracy: 0.9844 - val_loss: 0.2484 - val_accuracy: 0.9337
Epoch 118/300
213/213 [==============================] - 1s 6ms/step - loss: 0.0413 - accuracy: 0.9856 - val_loss: 0.2535 - val_accuracy: 0.9261
Epoch 119/300
213/213 [==============================] - 1s 6ms/step - loss: 0.0390 - accuracy: 0.9843 - val_loss: 0.2576 - val_accuracy: 0.9308
Epoch 120/300
213/213 [==============================] - 1s 5ms/step - loss: 0.0403 - accuracy: 0.9849 - val_loss: 0.2599 - val_accuracy: 0.9343
Epoch 121/300
213/213 [==============================] - 1s 5ms/step - loss: 0.0346 - accuracy: 0.9869 - val_loss: 0.2549 - val_ac

213/213 [==============================] - 1s 3ms/step - loss: 0.0274 - accuracy: 0.9884 - val_loss: 0.2649 - val_accuracy: 0.9313
Epoch 172/300
213/213 [==============================] - 1s 3ms/step - loss: 0.0328 - accuracy: 0.9888 - val_loss: 0.2521 - val_accuracy: 0.9325
Epoch 173/300
213/213 [==============================] - 1s 3ms/step - loss: 0.0208 - accuracy: 0.9925 - val_loss: 0.2453 - val_accuracy: 0.9337
Epoch 174/300
213/213 [==============================] - 1s 2ms/step - loss: 0.0331 - accuracy: 0.9872 - val_loss: 0.2004 - val_accuracy: 0.9425
Epoch 175/300
213/213 [==============================] - 1s 3ms/step - loss: 0.0150 - accuracy: 0.9953 - val_loss: 0.2168 - val_accuracy: 0.9437
Epoch 176/300
213/213 [==============================] - 1s 2ms/step - loss: 0.0297 - accuracy: 0.9902 - val_loss: 0.2543 - val_accuracy: 0.9349
Epoch 177/300
213/213 [==============================] - 1s 3ms/step - loss: 0.0271 - accuracy: 0.9903 - val_loss: 0.2013 - val_accuracy: 0.9437

213/213 [==============================] - 1s 2ms/step - loss: 0.0214 - accuracy: 0.9921 - val_loss: 0.2989 - val_accuracy: 0.9302
Epoch 228/300
213/213 [==============================] - 0s 2ms/step - loss: 0.0193 - accuracy: 0.9928 - val_loss: 0.2205 - val_accuracy: 0.9378
Epoch 229/300
213/213 [==============================] - 0s 2ms/step - loss: 0.0184 - accuracy: 0.9934 - val_loss: 0.2272 - val_accuracy: 0.9419
Epoch 230/300
213/213 [==============================] - 0s 2ms/step - loss: 0.0231 - accuracy: 0.9924 - val_loss: 0.2775 - val_accuracy: 0.9337
Epoch 231/300
213/213 [==============================] - 1s 2ms/step - loss: 0.0316 - accuracy: 0.9903 - val_loss: 0.2156 - val_accuracy: 0.9472
Epoch 232/300
213/213 [==============================] - 0s 2ms/step - loss: 0.0197 - accuracy: 0.9913 - val_loss: 0.2404 - val_accuracy: 0.9372
Epoch 233/300
213/213 [==============================] - 0s 2ms/step - loss: 0.0252 - accuracy: 0.9922 - val_loss: 0.2327 - val_accuracy: 0.9419

213/213 [==============================] - 0s 2ms/step - loss: 0.0231 - accuracy: 0.9932 - val_loss: 0.2325 - val_accuracy: 0.9378
Epoch 284/300
213/213 [==============================] - 0s 2ms/step - loss: 0.0120 - accuracy: 0.9959 - val_loss: 0.2026 - val_accuracy: 0.9489
Epoch 285/300
213/213 [==============================] - 0s 2ms/step - loss: 0.0087 - accuracy: 0.9966 - val_loss: 0.2323 - val_accuracy: 0.9384
Epoch 286/300
213/213 [==============================] - 0s 2ms/step - loss: 0.0120 - accuracy: 0.9960 - val_loss: 0.2277 - val_accuracy: 0.9437
Epoch 287/300
213/213 [==============================] - 0s 2ms/step - loss: 0.0154 - accuracy: 0.9952 - val_loss: 0.2374 - val_accuracy: 0.9413
Epoch 288/300
213/213 [==============================] - 0s 2ms/step - loss: 0.0201 - accuracy: 0.9947 - val_loss: 0.2757 - val_accuracy: 0.9384
Epoch 289/300
213/213 [==============================] - 0s 2ms/step - loss: 0.0169 - accuracy: 0.9941 - val_loss: 0.2600 - val_accuracy: 0.9372

In [175]:
test_model(model,np.array(feature2),np.array(y2),sc,pca,False)

acc:0.726552
[[207 152 117]
 [110 909  58]
 [ 68  81 441]]


0.7265515632291181

In [92]:
ind = ((y2==1)|(y2==2)|(y2==6))
oc = OneHotEncoder()
y_01 = oc.fit_transform(y2[ind,np.newaxis]).toarray()
y_pred=model.predict(sc.transform(np.array(feature2)[ind,:]))
ind2=np.where(np.argmax(y_pred,axis=1) != np.argmax(y_01,axis=1))
#ind2

In [170]:
model.save('./model/ann2.h5')

In [43]:
ind = np.array((y==0)|(y==1)|(y==2)|(y==6))
#feature
ind

array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

In [ ]:
#test_model(model,feature2_sc,np.array(y2))
ind = ((y==1)|(y==2)|(y==6))
y_01 = y[ind].copy()
oc = OneHotEncoder()
y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
pred=model.predict(feature_sc[ind,:])
np.argmax(pred,axis=1)
np.argmax(y_01,axis=1)
metrics.confusion_matrix(np.argmax(y_01,axis=1),np.argmax(pred,axis=1))

In [ ]:
#acc_ann = pd.DataFrame(acc,columns=['dnn1'],index=['train','valid','test','rest data'])
#acc_ann_com=pd.concat([acc_ann_com,acc_ann.T])
#acc_ann.loc['drop_mDWT',:]=[train,valid,test,rest]
acc_ann_com.loc['dnn2',:]=acc
acc_ann_com

# test feature from Left or Right

In [ ]:
input_ = layers.Input(shape=feature.loc[:,ind_temp].shape[1:])
l1 = layers.Dense(128,activation='elu')(input_)
drop1 = layers.Dropout(0.2)(l1)
l2 = layers.Dense(64,activation='elu')(drop1)
drop2 = layers.Dropout(0.2)(l2)
#l3 = layers.Dense(32,activation='elu')(drop2)
#drop3 = layers.Dropout(0.2)(l3)
#l4 = layers.Dense(16,activation='selu')(l3)
#drop4 = layers.Dropout(0.2)(l4)
output = layers.Dense(1,activation='sigmoid')(drop2)
model = Model(inputs=[input_],outputs=[output])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [209]:
acc={}
cols = ['LEFT','RIGHT']

#sc = StandardScaler(with_mean=True)

for col in cols:
    ind_temp=feature.columns.str.contains(col)
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    #acc[col] = [train,valid,test,acc_rest]
    print(acc_rest)

Train on 23125 samples, validate on 5782 samples
Epoch 1/200
23125/23125 [==============================] - 4s 167us/sample - loss: 1.9657 - accuracy: 0.5308 - val_loss: 1.3537 - val_accuracy: 0.6404
Epoch 2/200
23125/23125 [==============================] - 3s 109us/sample - loss: 1.3913 - accuracy: 0.6479 - val_loss: 1.1555 - val_accuracy: 0.7008
Epoch 3/200
23125/23125 [==============================] - 3s 110us/sample - loss: 1.2045 - accuracy: 0.7052 - val_loss: 1.0439 - val_accuracy: 0.7771
Epoch 4/200
23125/23125 [==============================] - 2s 105us/sample - loss: 1.1076 - accuracy: 0.7394 - val_loss: 0.9467 - val_accuracy: 0.7608
Epoch 5/200
23125/23125 [==============================] - 2s 86us/sample - loss: 1.0526 - accuracy: 0.7501 - val_loss: 0.9212 - val_accuracy: 0.7508
Epoch 6/200
23125/23125 [==============================] - 2s 93us/sample - loss: 0.9990 - accuracy: 0.7677 - val_loss: 0.8937 - val_accuracy: 0.7975
Epoch 7/200
23125/23125 [======================

23125/23125 [==============================] - 2s 71us/sample - loss: 0.8517 - accuracy: 0.8045 - val_loss: 0.8904 - val_accuracy: 0.8210
Epoch 9/200
23125/23125 [==============================] - 2s 71us/sample - loss: 0.8091 - accuracy: 0.8131 - val_loss: 1.0212 - val_accuracy: 0.8426
Epoch 10/200
23125/23125 [==============================] - 2s 72us/sample - loss: 0.7750 - accuracy: 0.8260 - val_loss: 1.0699 - val_accuracy: 0.8438
Epoch 11/200
23125/23125 [==============================] - 2s 72us/sample - loss: 0.7591 - accuracy: 0.8365 - val_loss: 0.9963 - val_accuracy: 0.8513
Epoch 12/200
23125/23125 [==============================] - 2s 70us/sample - loss: 0.7071 - accuracy: 0.8416 - val_loss: 1.0759 - val_accuracy: 0.8464
Epoch 13/200
23125/23125 [==============================] - 2s 71us/sample - loss: 0.6955 - accuracy: 0.8444 - val_loss: 1.0100 - val_accuracy: 0.8442
Epoch 14/200
23125/23125 [==============================] - 2s 71us/sample - loss: 0.7017 - accuracy: 0.8478

In [ ]:
acc_lr=pd.DataFrame(acc,index=['train','valid','test','rest data'])#.to_csv('./results/acc_lr_ann.csv')
#acc_com=pd.concat([acc_ann_com.drop(['dnn','dnn1'],'index'),acc_lr.T])
acc_com

# test features from 2 of 8 signals

In [ ]:
#test features from 2 of 8 signals

acc_2f={}
cols = [ 'LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF',
        'RIGHT_TA','RIGHT_TS','RIGHT_BF', 'RIGHT_RF']


for p in combinations(cols[:4],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)
    
for p in combinations(cols[4:],2):
    ind_temp=feature.columns.str.contains(p[0])| feature.columns.str.contains(p[1])
    #feature_sc = sc.fit_transform(feature.loc[:,ind_temp])
    #feature2_sc = sc.transform(feature2.loc[:,ind_temp])
    train,valid,test,sc = train_model(model,np.array(feature.loc[:,ind_temp]),np.array(y))
    acc_rest=test_model(model,np.array(feature2.loc[:,ind_temp]),np.array(y2),sc)
    acc_2f[p[0]+'_'+p[1]] = [train,valid,test,acc_rest]
    print(acc_rest)

In [ ]:
#acc_2f_ann = pd.DataFrame(acc_2f,index=['train','valid','test','rest data']).T
#acc_com = pd.concat([acc_com,acc_2f_ann])
acc_com.to_csv('./results/dropna/acc_com_ann.csv')

In [ ]:
pd.DataFrame(acc_2f,index=['train','valid','test','rest data']).T

# Test on some of rest

In [ ]:
some_of_rest = ['正常/P940_MSham_B_Walking_trial_6_emg.csv',
                '正常/P940_M050_B_Walking_trial_4_emg.csv',
                '正常/P812_M100_A_Walking_trial_3_emg.csv',
                '正常/P645_M050_A_Walking_trial_3_emg.csv',
                '正常/P623_Msham_B_Walking_trial_2_emg.csv',
                '正常/P551_M50_B_Walking_trial_6_emg.csv',
                'P379_M050_2_OFF_A_FoG_trial_1_emg.csv',
                'P551_M050_2_B_FoG_trial_2_emg.csv']
#booster = xgb.Booster()
#booster.load_model('./model/XGBoost_W256_S64_Left.json')
#model = xgb.XGBClassifier()
#model._Booster = booster
acc = []
columns=['LEFT_TA','LEFT_TS','LEFT_BF', 'LEFT_RF']
for file in some_of_rest:
    path = './data/'+file
    feature2,y2 = dp.pipeline_feature(path,width=256,stride=64,scaler=False,
                                      threshold_WAMP=threshold_WAMP,
                                      threshold_ZC=threshold_ZC,
                                      threshold_SSC=threshold_SSC,
                                      bins=bins,
                                      ranges=HIST_range,
                                      show_para=False,
                                      filt = 250)
    feature2_sc = sc.transform(feature2)
    acc += [test_model(model,feature2_sc,y2)]
print(acc)

In [ ]:
y_pred=model.predict(feature2_sc)
metrics.accuracy_score(y2,y_pred>0.5)

## RNN

In [92]:
files = np.array(df.columns)
X = []
Y = []
#sc = StandardScaler(with_mean=False)
i = 0
start = time.time()
for file in files:
    emg_data = pd.read_csv('./data/'+file)
    #emg_data = emg_data.fillna({'LEFT_TA':emg_data.LEFT_TA.mean(),
    #                            'LEFT_TS':emg_data.LEFT_TS.mean(),
    #                            'LEFT_BF':emg_data.LEFT_BF.mean(),
    #                            'LEFT_RF':emg_data.LEFT_RF.mean(),
    #                            'RIGHT_TA':emg_data.RIGHT_TA.mean(),
    #                            'RIGHT_TS':emg_data.RIGHT_TS.mean(),
    #                            'RIGHT_BF':emg_data.RIGHT_BF.mean(),
    #                            'RIGHT_RF':emg_data.RIGHT_RF.mean()})
    #emg_data = emg_data[emg_data.Label1 == emg_data.Label2].reset_index(drop=True)
    emg_data = emg_data.dropna().reset_index(drop=True)
    x = np.array(emg_data.iloc[:,3:])
    y = np.array(emg_data.Label2)
    #x,y = dp.generate_window_slide_data(emg_data,width=width,stride=stride,scaler=False,same_label=True)
    X += x.tolist()
    Y += y.tolist()
    i += 1
    print('i:%d'%i,x.shape)
end = time.time()
duration = end-start
print('Duration: %f'%(duration))

i:1 (16583, 8)
i:2 (4489, 8)
i:3 (35121, 8)
i:4 (31027, 8)
i:5 (30783, 8)
i:6 (159539, 8)
i:7 (225455, 8)
i:8 (362273, 8)
i:9 (28868, 8)
i:10 (20878, 8)
i:11 (25322, 8)
i:12 (24242, 8)
i:13 (21728, 8)
i:14 (27916, 8)
i:15 (27146, 8)
i:16 (25947, 8)
i:17 (26203, 8)
i:18 (89500, 8)
i:19 (70282, 8)
i:20 (79595, 8)
i:21 (73280, 8)
Duration: 4.539805


In [10]:
def train_model(model,feature,y,binary=True,file=None):
    if binary:
        ind = ((y==0)|(y==1)|(y==2)|(y==3)|(y==4)|(y==6))
        ind1 = ((y==4)|(y==1)|(y==2)|(y==3)|(y==6))
        y_01 = y.copy()
        #ind1 = ((y_01==1)|(y_01==2)|(y_01==6))
        y_01[ind1] = 1
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:1,1:5}
    else:
        ind = ((y==1)|(y==2)|(y==6))
        #ind = ((y==1)|(y==6))
        y_01 = y[ind].copy()
        oc = OneHotEncoder()
        y_01 = oc.fit_transform(np.array(y_01)[:,np.newaxis]).toarray()
        cw = None#{0:5,1:1,2:1}#{0:2,1:1,2:10,3:2}
    x_full,x_test,y_full,y_test = train_test_split(np.array(feature)[ind,:],y_01,
                                                   test_size=0.2,
                                                   random_state=123,
                                                   shuffle=True)
    x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,
                                                       test_size=0.25,
                                                       random_state=555,
                                                       shuffle=True)
    
    #sm = BorderlineSMOTE(random_state=50,kind='borderline-1')
    #sm = SMOTE(random_state=50)
    #print(y_full.shape)
    #x_full,y_full = sm.fit_resample(x_full,y_full)
    #print(y_full_n.shape)
    x_full = np.reshape(x_full,(-1,128*8))
    sc = StandardScaler(with_mean=False)
    #sc = MinMaxScaler((0,1))
    x_train = sc.fit_transform(x_full)
    x_train = x_train.reshape((-1,128,8))
    #pca = PCA(n_components=150)
    #x_train = pca.fit_transform(x_train)
    #x_valid = sc.transform(x_valid)
    x_test = np.reshape(x_test,(-1,128*8))
    x_test = sc.transform(x_test)
    x_test = x_test.reshape((-1,128,8))
    #x_test = pca.transform(x_test)
    #x_train = x_full
    
    early_stopping = callbacks.EarlyStopping(patience = 10,
                                             monitor = 'val_accuracy', 
                                             restore_best_weights=True)
    history = model.fit(x_train,y_full,validation_data=[x_test,y_test],
                        epochs=300,batch_size=500,class_weight=cw,
                        callbacks=[early_stopping],
                        shuffle=True)
    
    if binary:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #test = metrics.accuracy_score(y_test,y_pred_t>0.5)
        
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(y_valid,np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))
        #train = metrics.accuracy_score(y_full,y_pred_ta>0.5)
        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
        
        #print('train: \n',metrics.confusion_matrix(y_full,y_pred_ta>0.5))
        #print('test: \n',metrics.confusion_matrix(y_test,y_pred_t>0.5))

    else:
        y_pred_t=model.predict(x_test)
        test = metrics.accuracy_score(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1))
        #y_pred_v=model.predict(x_valid)
        #valid = metrics.accuracy_score(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1))
        y_pred_ta=model.predict(x_train)
        train = metrics.accuracy_score(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1))        
        print('train: \n',metrics.confusion_matrix(np.argmax(y_full,axis=1),np.argmax(y_pred_ta,axis=1)))
        #print('valid: \n',metrics.confusion_matrix(np.argmax(y_valid,axis=1),np.argmax(y_pred_v,axis=1)))
        print('test: \n',metrics.confusion_matrix(np.argmax(y_test,axis=1),np.argmax(y_pred_t,axis=1)))
    print('test:%f'%test)
    #print('valid:%f'%valid)
    print('train:%f'%train)
    if file != None:
        model.save_model(file)
    return train,test,sc#,pca

In [ ]:
feature_re = feature_sc.reshape((-1,X.shape[1],1))
x_full,x_test,y_full,y_test = train_test_split(np.array(feature_re),np.array(y_01),test_size=0.2,random_state=123)
x_train,x_valid,y_train,y_valid = train_test_split(x_full,y_full,test_size=0.2,random_state=555)

In [11]:
input_ = layers.Input(shape=[128,8])
lstm1 = layers.LSTM(50,return_sequences=True)(input_)
drop1 = layers.Dropout(0.2)(lstm1)
lstm2 = layers.LSTM(50,return_sequences=True)(drop1)
drop2 = layers.Dropout(0.2)(lstm2)
#lstm3 = layers.LSTM(50,return_sequences=True)(drop2)
#drop3 = layers.Dropout(0.2)(lstm3)
lstm4 = layers.LSTM(50)(drop2)
drop4 = layers.Dropout(0.2)(lstm4)
dense1 = layers.Dense(64,activation='relu')(drop4)
drop5 = layers.Dropout(0.2)(dense1)
dense2 = layers.Dense(32,activation='relu')(drop5)
output = layers.Dense(3,activation='softmax')(dense2)
model = Model(inputs=[input_],outputs=[output])

In [12]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
early_stopping = callbacks.EarlyStopping(patience = 20,
                                         monitor = 'val_accuracy', 
                                         restore_best_weights=True)
history = model.fit(x_train,y_train,validation_data=[x_valid,y_valid],
                    epochs=100,batch_size=500,
                   callbacks=[early_stopping])

In [13]:
train,test,sc = train_model(model,np.array(x),np.array(y),False)

Train on 8872 samples, validate on 2219 samples
Epoch 1/300
8872/8872 [==============================] - 88s 10ms/sample - loss: 0.9841 - accuracy: 0.4876 - val_loss: 0.8857 - val_accuracy: 0.5002
Epoch 2/300
8872/8872 [==============================] - 82s 9ms/sample - loss: 0.8572 - accuracy: 0.5571 - val_loss: 0.8288 - val_accuracy: 0.6471
Epoch 3/300
8872/8872 [==============================] - 81s 9ms/sample - loss: 0.8051 - accuracy: 0.6443 - val_loss: 0.7573 - val_accuracy: 0.6670
Epoch 4/300
8872/8872 [==============================] - 81s 9ms/sample - loss: 0.7433 - accuracy: 0.6677 - val_loss: 0.7142 - val_accuracy: 0.6791
Epoch 5/300
8872/8872 [==============================] - 81s 9ms/sample - loss: 0.7212 - accuracy: 0.6787 - val_loss: 0.6996 - val_accuracy: 0.6886
Epoch 6/300
8872/8872 [==============================] - 87s 10ms/sample - loss: 0.7071 - accuracy: 0.6878 - val_loss: 0.6952 - val_accuracy: 0.7008
Epoch 7/300
8872/8872 [==============================] - 81s 9

KeyboardInterrupt: 

In [ ]:
feature2_re = feature2_sc.reshape((-1,feature.shape[1],1))

In [ ]:
test_model(model,x_test,y_test)

In [113]:
cost_matrix=np.array([[0,1,1,1],
                  [1,0,1,1],
                  [10,100,0,10],
                  [1.,1.,1,0]])
cost_matrix=np.array([[0,2.],
                  [5.,0]])

In [58]:
P0 = 55080/(55080+8181)
P1 = 8181/(55080+8181)
P = [P0,P1]
cost_vec = np.zeros(2)
for i in range(2):
    for j in range(2):
        if i == j:
            continue
        cost_vec[i]+=1/(1-P[i])*P[j]*cost_matrix[i,j]

In [59]:
cost_vec

array([2., 5.])

In [112]:
tf.transpose(t)

<tf.Tensor: id=1567100, shape=(4, 4), dtype=float64, numpy=
array([[  0.,   1.,  10.,   1.],
       [  1.,   0., 100.,   1.],
       [  1.,   1.,   0.,   1.],
       [  1.,   1.,  10.,   0.]])>

In [116]:
np.reshape(cost_matrix,(-1,1))

array([[0.],
       [2.],
       [5.],
       [0.]])